Making sql tables: one with addresses and one with species/urlPaths

In [2]:
import pandas as pd
import sqlite3
import os

address_path = os.path.join('..', 'dev', 'Cleaned_Street_Tree_list.csv')
addresses = pd.read_csv(address_path, index_col=0)
addresses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174480 entries, 121399 to 15238
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   qSpecies   174480 non-null  object
 1   qAddress   174480 non-null  object
 2   SiteOrder  174480 non-null  int64 
 3   qSiteInfo  174480 non-null  object
dtypes: int64(1), object(3)
memory usage: 6.7+ MB


In [124]:
tree_list = addresses    
tree_list.qAddress = tree_list.qAddress.str.lower()

split_street_names = tree_list.qAddress.str.split(' ', n=1, expand=True).rename({0: 'street_number', 1: 'street_name'},
                                                                              axis=1) 
split_street_names.street_number = split_street_names.street_number.str.extract(r'(^[0-9]+)', expand=False)
split_street_names.street_name = split_street_names.street_name.dropna().str.replace(r'\b(revised|\(revised\))\b', '').str.strip()
tree_list.qAddress = split_street_names.street_number.str.cat(split_street_names.street_name, ' ')
tree_list

C:\Users\Prog\AppData\Local\Temp\ipykernel_15700\2050067751.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  split_street_names.street_name = split_street_names.street_name.dropna().str.replace(r'\b(revised|\(revised\))\b', '').str.strip()


,qAddress,qSpecies,SiteOrder
TreeID,,,
121399,349 cargo way,Corymbia ficifolia :: Red Flowering Gum,1
251210,1170 ingalls st,Eucalyptus polyanthemos :: Silver Dollar Eucal...,2
255131,401 brotherhood way,Corymbia ficifolia :: Red Flowering Gum,9
255229,300 goettingen st,Lophostemon confertus :: Brisbane Box,4
251438,2501 santiago st,Cupressus macrocarpa :: Monterey Cypress,1
...,...,...,...
231,2975 19th ave,Platanus x hispanica :: Sycamore: London Plane,2
92652,2201 baker st,Azara microphylla :: Little-Leaf Azara,18
108942,115 prospect ave,Arbutus 'Marina' :: Hybrid Strawberry Tree,1


In [3]:
addresses = addresses.loc[:,['qAddress','qSpecies', 'SiteOrder']].astype({'qAddress': 'string', 'SiteOrder': 'uint16'})
addresses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174480 entries, 121399 to 15238
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   qAddress   174480 non-null  string
 1   qSpecies   174480 non-null  object
 2   SiteOrder  174480 non-null  uint16
dtypes: object(1), string(1), uint16(1)
memory usage: 4.3+ MB


In [15]:
address_species = set(addresses.qSpecies.tolist())

In [81]:
%%timeit
addresses.loc[addresses.qAddress.str.lower() == '1470 valencia st', 'qSpecies'].item()

44.7 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
species_path = os.path.join('..','dev','mapped_species.csv' )
species = pd.read_csv(species_path, index_col=0)
species.head()

,qSpecies,urlPath
0,Corymbia ficifolia :: Red Flowering Gum,540
1,Eucalyptus polyanthemos :: Silver Dollar Eucal...,561
2,Lophostemon confertus :: Brisbane Box,1425
3,Cupressus macrocarpa :: Monterey Cypress,476
4,Jacaranda mimosifolia :: Jacaranda,741


In [19]:
species_with_keys = set(species.qSpecies.tolist())
address_species.difference(species_with_keys)

set()

In [191]:
species_dict = species.reset_index().set_index('qSpecies').drop('urlPath', axis=1).to_dict()['index']
species_dict

{'Corymbia ficifolia :: Red Flowering Gum': 0,
 'Eucalyptus polyanthemos :: Silver Dollar Eucalyptus': 1,
 'Lophostemon confertus :: Brisbane Box': 2,
 'Cupressus macrocarpa :: Monterey Cypress': 3,
 'Jacaranda mimosifolia :: Jacaranda': 4,
 'Acacia decurrens :: Acacia: Silver Wattle': 5,
 'Pyrus kawakamii :: Evergreen Pear': 6,
 'Melaleuca spp :: Melaleuca spp': 7,
 'Eriobotrya japonica :: Edible Loquat': 8,
 'Platanus x hispanica :: Sycamore: London Plane': 9,
 "Acacia baileyana :: Bailey's Acacia": 10,
 'Prunus cerasifera :: Cherry Plum': 11,
 'Pinus Spp :: Pine Spp': 12,
 'Pinus radiata :: Monterey Pine': 13,
 'Eucalyptus sideroxylon :: Red Ironbark': 14,
 'Acacia melanoxylon :: Blackwood Acacia': 15,
 'Lyonothamnus floribundus subsp. asplenifolius :: Santa Cruz Ironwood': 16,
 'Acer spp :: Maple': 17,
 'Pittosporum undulatum :: Victorian Box': 18,
 'Eucalyptus globulus :: Blue Gum': 19,
 'Acacia longifolia :: Golden Wattle': 20,
 'Eucalyptus Spp :: Eucalyptus': 21,
 'Pinus pinea :

In [192]:
addresses = addresses.replace({'qSpecies': species_dict})

In [193]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174962 entries, 121399 to 15238
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   qAddress   174962 non-null  string
 1   qSpecies   174962 non-null  int64 
 2   SiteOrder  174962 non-null  uint16
dtypes: int64(1), string(1), uint16(1)
memory usage: 4.3 MB


In [198]:
addresses = addresses.astype({'qSpecies': 'uint16'})
addresses.qAddress = addresses.qAddress.str.lower()
addresses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174962 entries, 121399 to 15238
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   qAddress   174962 non-null  string
 1   qSpecies   174962 non-null  uint16
 2   SiteOrder  174962 non-null  uint16
dtypes: string(1), uint16(2)
memory usage: 3.3 MB


In [74]:
import timeit

In [88]:
%%timeit
species_key = addresses.loc[addresses.qAddress.str.lower() == '1470 valencia st', 'qSpecies'].item()
species.iloc[species_key].qSpecies

47.9 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [199]:
import sqlite3
with sqlite3.connect('SF_trees.db') as con:
    addresses.to_sql('addresses', con, if_exists='replace')
    species.to_sql('species', con, if_exists='replace')

In [214]:
%%timeit
query = """
SELECT qSpecies
FROM addresses
WHERE qAddress = '1470 valencia st'"""

def species_query(key):
    new_query = f"""
    SELECT qSpecies
    FROM species
    WHERE "index" = {key}"""
    return new_query

with sqlite3.connect('SF_trees.db') as con:
    a = pd.read_sql(query, con).qSpecies
    if a.size > 0:
        a = a.item()
        specie = pd.read_sql(species_query(a), con)
        specie.qSpecies.item()
    else:
        pass

21.8 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
split_streets = addresses.qAddress.str.split(' ', n=1, expand=True).rename({0: 'street_number', 1: 'street_name'}, axis=1)
split_addresses = addresses.join(split_streets).loc[:,['street_number', 'street_name', 'qSpecies', 'SiteOrder']]
split_addresses.street_name = split_addresses.street_name.str.lower()
split_addresses

,street_number,street_name,qSpecies,SiteOrder
TreeID,,,,
121399,349X,cargo way,Corymbia ficifolia :: Red Flowering Gum,1
251210,1170,ingalls st,Eucalyptus polyanthemos :: Silver Dollar Eucal...,2
255131,401X,brotherhood way,Corymbia ficifolia :: Red Flowering Gum,9
255229,300X,goettingen st,Lophostemon confertus :: Brisbane Box,4
251438,2501X,santiago st,Cupressus macrocarpa :: Monterey Cypress,1
...,...,...,...,...
231,2975,19th ave,Platanus x hispanica :: Sycamore: London Plane,2
92652,2201,baker st,Azara microphylla :: Little-Leaf Azara,18
108942,115B,prospect ave,Arbutus 'Marina' :: Hybrid Strawberry Tree,1


In [115]:
split_addresses.street_number = split_streets.street_number.str.extract(r"(^[0-9]+)", expand=False)

In [94]:
extracted.loc[extracted.isna()]

TreeID
50817    <NA>
Name: street_number, dtype: string

In [96]:
split_streets

,street_number,street_name
TreeID,,
121399,349,Cargo Way
251210,1170,Ingalls St
255131,401,Brotherhood Way
255229,300,Goettingen St
251438,2501,Santiago St
...,...,...
231,2975,19th Ave
92652,2201,Baker St
108942,115,Prospect Ave


In [97]:
addresses_without_letters = split_streets.street_number.loc[~split_streets.street_number.str.contains(r'[0-9]$')].str[:-1]

In [98]:
addresses_without_letters

Series([], Name: street_number, dtype: string)

In [62]:
split_streets.update(addresses_without_letters)

C:\Users\Prog\AppData\Local\Temp\ipykernel_15700\3627194058.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  split_streets.update(addresses_without_letters)


In [63]:
split_streets.head()

,street_number,street_name
TreeID,,
121399,349,Cargo Way
251210,1170,Ingalls St
255131,401,Brotherhood Way
255229,300,Goettingen St
251438,2501,Santiago St


In [114]:
split_streets.street_number.loc[~split_streets.street_number.str.contains(r'[0-9]$')]

Series([], Name: street_number, dtype: string)

In [112]:
split_addresses.street_name = split_addresses.street_name.dropna().str.replace(r'\b(revised|\(revised\))\b', '').str.strip()
split_addresses.loc[split_addresses.street_name == street_names[944], ['street_number', 'qSpecies', 'SiteOrder']]

C:\Users\Prog\AppData\Local\Temp\ipykernel_15700\3410652132.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  split_addresses.street_name = split_addresses.street_name.dropna().str.replace(r'\b(revised|\(revised\))\b', '').str.strip()


,street_number,qSpecies,SiteOrder
TreeID,,,
178621,2000,Platanus x hispanica :: Sycamore: London Plane,2
852,366,Acer rubrum :: Red Maple,1
180087,2100x,Tristaniopsis laurina :: Swamp Myrtle,10
265495,2700,Ligustrum lucidum :: Glossy Privet,1
253080,2849X,Eriobotrya deflexa :: Bronze Loquat,13
...,...,...,...
180073,1900x,Tristaniopsis laurina :: Swamp Myrtle,45
122857,3000X,Tristaniopsis laurina :: Swamp Myrtle,4
1073,2118,Platanus x hispanica :: Sycamore: London Plane,1


In [118]:
split_addresses['qAddress'] = split_addresses.street_number.str.cat(split_addresses.street_name, ' ')
split_addresses

,street_number,street_name,qSpecies,SiteOrder,qAddress
TreeID,,,,,
121399,349,cargo way,Corymbia ficifolia :: Red Flowering Gum,1,349 cargo way
251210,1170,ingalls st,Eucalyptus polyanthemos :: Silver Dollar Eucal...,2,1170 ingalls st
255131,401,brotherhood way,Corymbia ficifolia :: Red Flowering Gum,9,401 brotherhood way
255229,300,goettingen st,Lophostemon confertus :: Brisbane Box,4,300 goettingen st
251438,2501,santiago st,Cupressus macrocarpa :: Monterey Cypress,1,2501 santiago st
...,...,...,...,...,...
231,2975,19th ave,Platanus x hispanica :: Sycamore: London Plane,2,2975 19th ave
92652,2201,baker st,Azara microphylla :: Little-Leaf Azara,18,2201 baker st
108942,115,prospect ave,Arbutus 'Marina' :: Hybrid Strawberry Tree,1,115 prospect ave


In [106]:
for i, street_name in enumerate(street_names):
    if street_name.find('revised') != -1:
        print(street_name)

In [224]:
import json
street_name_list = street_names.tolist()
with open('street_names.json', 'w') as fp:
    json.dump(street_name_list, fp)

In [226]:
%%timeit
with open('street_names.json', 'r') as fp:
    json.load(fp)

482 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [206]:
with sqlite3.connect('SF_trees_by_street.db') as con:
    for street_name in street_names:
        street_table = split_addresses.loc[
            split_addresses.street_name == street_name, ['street_number', 'qSpecies', 'SiteOrder']]
        if not street_table.empty:
            street_table.to_sql(street_name, con, if_exists='replace')
        

In [210]:
with sqlite3.connect('SF_trees_by_street.db') as con:
    species.to_sql('species', con, if_exists='replace')

In [212]:
%%timeit
query = """
SELECT qSpecies
FROM "valencia st"
WHERE street_number = 1470"""

def species_query(key):
    new_query = f"""
    SELECT qSpecies
    FROM species
    WHERE "index" = {key}"""
    return new_query

with sqlite3.connect('SF_trees_by_street.db') as con:
    a = pd.read_sql(query, con).qSpecies
    if a.size > 0:
        a = a.item()
        specie = pd.read_sql(species_query(a), con)
        specie.qSpecies.item()
    else:
        pass

16.8 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
addresses.loc[addresses.qAddress.str.contains('X')]

,qSpecies,qAddress,SiteOrder,qSiteInfo
TreeID,,,,
121399,Corymbia ficifolia :: Red Flowering Gum,349X Cargo Way,1,Sidewalk: Curb side : Cutout
255131,Corymbia ficifolia :: Red Flowering Gum,401X Brotherhood Way,9,Sidewalk: Property side : Yard
255229,Lophostemon confertus :: Brisbane Box,300X Goettingen St,4,Sidewalk: Curb side : Cutout
251438,Cupressus macrocarpa :: Monterey Cypress,2501X Santiago St,1,Median : Yard
20353,Acacia decurrens :: Acacia: Silver Wattle,262X Sunset Blvd,3,Sidewalk: Curb side : Cutout
...,...,...,...,...
99678,Ligustrum lucidum :: Glossy Privet,1250X Shrader St,9,Sidewalk: Curb side : Cutout
109994,Ligustrum lucidum :: Glossy Privet,501X Shotwell St,11,Sidewalk: Curb side : Cutout
97823,Prunus serrulata :: Ornamental Cherry,595X Liberty St,2,Sidewalk: Curb side : Cutout
